In [1]:
import sys
import os
sys.path.append(os.path.abspath('autoformer'))
from autoformer.model import Autoformer
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

2025-09-18 22:18:45.159536: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758221325.297329    4348 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758221325.334101    4348 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758221325.629417    4348 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758221325.629445    4348 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758221325.629447    4348 computation_placer.cc:177] computation placer alr

In [2]:
def create_time_windows(data, target, window_size, horizon=1):
    X_windows = []
    y_windows = []
    for i in range(len(data) - window_size - horizon + 1):
        X_windows.append(data[i:i+window_size])  # این باید یک sequence باشد
        y_windows.append(target[i+window_size:i+window_size+horizon])
    return np.array(X_windows), np.array(y_windows)

In [3]:
df= pd.read_csv('data/ETTh1.csv')
df['date']= pd.to_datetime(df['date'])
df['timestamp'] = df['date'].astype('int')
df = df.set_index(['date'])
df.head()

,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT,timestamp
date,,,,,,,,
2016-07-01 00:00:00,5.827,2.009,1.599,0.462,4.203,1.340,30.531000,1467331200000000000
2016-07-01 01:00:00,5.693,2.076,1.492,0.426,4.142,1.371,27.787001,1467334800000000000
2016-07-01 02:00:00,5.157,1.741,1.279,0.355,3.777,1.218,27.787001,1467338400000000000
2016-07-01 03:00:00,5.090,1.942,1.279,0.391,3.807,1.279,25.044001,1467342000000000000
2016-07-01 04:00:00,5.358,1.942,1.492,0.462,3.868,1.279,21.948000,1467345600000000000


In [4]:
X= df.drop(columns=['OT'])
X= X.values
y= df['OT'].values

In [5]:
scaler= StandardScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y.reshape(-1, 1)).flatten()

In [6]:
window_size = 24  # last 24 hour
horizon = 1  # forecast next hour

X_windows, y_windows = create_time_windows(X_scaled, y_scaled, window_size, horizon)

if len(y_windows.shape) == 1:
    y_windows = y_windows.reshape(-1, 1)


In [7]:
x_train, x_test, y_train, y_test= train_test_split(X_windows, y_windows, train_size=0.8, shuffle=False)
x_valid, x_test, y_valid, y_test= train_test_split(x_test, y_test, train_size=0.5, shuffle=False)

In [8]:
input_shape = (window_size, X.shape[1])
inputs= layers.Input(shape=input_shape)
x= Autoformer(d_out= 1,
            d_model= 16,
            n_heads= 4,
            conv_filter= 16,
            num_decoder=1,
            num_encoder=1)(inputs)

model= Model(inputs= inputs, outputs= x)      
model.compile(optimizer='adam', loss='mse')
model.summary()  

I0000 00:00:1758221330.680261    4348 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3209 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1
/home/amir/anaconda3/lib/python3.12/site-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'encoder', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 24, 7)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ autoformer (Autoformer)         │ (None, 24, 1)          │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.fit(
    x_train,
    y_train,
    validation_data=(x_valid, y_valid),
    shuffle=False,
    epochs=100,
)

Epoch 1/100
Instructions for updating:
Use fn_output_signature instead


/home/amir/anaconda3/lib/python3.12/site-packages/keras/src/layers/layer.py:1383: UserWarning: Layer 'auto_correlation_wrapper_3' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: ''tf.function only supports singleton tf.Variables created on the first call. Make sure the tf.Variable is only created once or created outside tf.function. See https://www.tensorflow.org/guide/function#creating_tfvariables for more information.''
  warnings.warn(
/home/amir/anaconda3/l

 31/435 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 2.5288

I0000 00:00:1758221346.586422    4435 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


435/435 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.9804

/home/amir/anaconda3/lib/python3.12/site-packages/keras/src/layers/layer.py:1383: UserWarning: Layer 'auto_correlation_wrapper_9' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: ''tf.function only supports singleton tf.Variables created on the first call. Make sure the tf.Variable is only created once or created outside tf.function. See https://www.tensorflow.org/guide/function#creating_tfvariables for more information.''
  warnings.warn(
/home/amir/anaconda3/l

435/435 ━━━━━━━━━━━━━━━━━━━━ 37s 48ms/step - loss: 0.9797 - val_loss: 0.7405
Epoch 2/100
435/435 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 2.2752 - val_loss: 0.8063
Epoch 3/100
435/435 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 1.9760 - val_loss: 0.8068
Epoch 4/100
435/435 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 1.7124 - val_loss: 0.7837
Epoch 5/100
435/435 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 1.6754 - val_loss: 0.7664
Epoch 6/100
435/435 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 1.6116 - val_loss: 0.7507
Epoch 7/100
435/435 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 1.5453 - val_loss: 0.7449
Epoch 8/100
435/435 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 1.4725 - val_loss: 0.7287
Epoch 9/100
435/435 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 1.4396 - val_loss: 0.7331
Epoch 10/100
435/435 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 1.3736 - val_loss: 0.7207
Epoch 11/100
435/435 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 1.3441 - val_loss: 0.7100
Epoch 12/100
435/435 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/st

In [10]:
# Evaluate the model accuracy on the validation set.
score = model.evaluate(x_test, y_test)

55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1328
